In [26]:
class Scalar:

    def __init__(self, data, prev_scalars=(), op=''):
        self.data = data
        self._prev_scalars = set(prev_scalars)
        
        self._op = op
        self.grad = 0
        self._backward = lambda: None
        
    def backward(self):
        
        def build_topo_order(scalar, topo_order):
            for prev_scalar in scalar._prev_scalars:
                if prev_scalar not in topo_order:
                    build_topo_order(prev_scalar, topo_order)
            topo_order.append(scalar)
            
        topo_order = []
        build_topo_order(self, topo_order)
        
        self.grad = 1
        for scalar in reversed(topo_order):
            scalar._backward()
        
    def __repr__(self):
        return f'Scalar[{self.data}, {self._op}]'
        
    def __add__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data + other.data, (self, other), '<Add>')
        
        def backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = backward

        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data * other.data, (self, other), '<Mul>')
        
        def backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = backward

        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Scalar(self.data ** other, (self,), f'<Pow{other}>')
        
        def backward():
            self.grad += out.grad * other * self.data**(other-1)
            
        out._backward = backward
        
        return out
    
    def __neg__(self):
        return self * -1
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return other + (-self)
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __rtruediv__(self, other):
        return other * self**-1
        

In [27]:
a = Scalar(2)
b = Scalar(3)

x = 2*a + 3*b
y = 5*(a**2) + 3*(b**3)

z = 2*x + 3*y
print(z)

z.backward()

print(a.grad, b.grad)

Scalar[329, <Add>]
64 249


In [29]:
class Tensor:
    
    def __init__(self, data, shape=None, prev_tensors=(), op=''):
        self.data = data
        
        if shape is None:
            self.shape = (len(data), )
        else:
            self.shape = shape
        
        self._prev_tensors = set(prev_tensors)
        
        self._op = op
        self.grad = 0
        self._backward = lambda: None
                
    def __repr__(self):
        return f'Tensor[{self.data}, {self._op}]'
    
    def get_shape(self):
        return f'Tensorshape{self.shape}'
    
    def broadcast(self, desired_shape):
        
        if self.shape == desired_shape:
            return self
        
        assert len(self.shape) <= len(desired_shape), "can only broadcast to higher dimensions"
        
        new_data = self.data
        new_shape = self.shape
        
        for i in range(len(desired_shape) - len(self.shape)):
            new_data = [new_data]
            new_shape = (1,) + new_shape
            
        for i in range(len(desired_shape) - len(self.shape)):
            new_data = new_data[0]
            new_shape = new_shape[1:]
            
        for i in range(len(desired_shape)):
            if new_shape[i] == 1 and desired_shape[i] != 1:
                new_data = [new_data] * desired_shape[i]
                new_shape = (desired_shape[i], )
                
        return Tensor(new_data, desired_shape, (self,), '<Broadcast>')

In [30]:
a = Tensor([1, 2, 3])
b = Tensor([4, 5, 6])

a = a.broadcast((3, 3))
print(a)

IndexError: tuple index out of range

In [2]:
import torch

In [9]:
a = torch.Tensor([1, 2, 3]).requires_grad_()
b = torch.Tensor([4, 5, 6]).requires_grad_()

x = 2*a + 3*b
y = 5*(a**2) + 3*(b**3)

z = 2*x + 3*y
r = z.sum()

r.backward()

print(a.grad, b.grad)

tensor([34., 64., 94.]) tensor([438., 681., 978.])
